## CIND820 Big Data Analytics Project - Data Mining for Customer Behavior Analysis in Retail Sales

In [39]:
# import libraries 

import pandas as pd
import numpy as np
import matplotlib as plt

In [40]:
# import dataset

path = r"C:\Users\jrroz\Desktop\June\TMU\Course Materials\CIND820\Data\Online Retail.xlsx"

df = pd.read_excel(path)


### 1. Dataset Inspection

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [42]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [43]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [44]:
# count of unique values

num_rows = df.shape[0]
num_cols = df.shape[1]
num_customers = df['CustomerID'].nunique()
num_products = df['StockCode'].nunique()
num_invoices = df['InvoiceNo'].nunique()
num_countries = df['Country'].nunique()

print(num_rows, num_cols, num_customers, num_products, num_invoices, num_countries)

541909 8 4372 4070 25900 38


In [45]:
# check data types 

df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [46]:
# change CustomerID data type

df['CustomerID'] = df['CustomerID'].astype("object")

df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

In [47]:
# check start and end dates

print(df['InvoiceDate'].min())
print(df['InvoiceDate'].max())

2010-12-01 08:26:00
2011-12-09 12:50:00


In [48]:
# descriptive statistics for quantitative columns

df[['Quantity', 'UnitPrice']].describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


### 2. Data Quality Check

2.1 Check for negative UnitPrice and negative Quantity
 
- 2.1.1 Flag "adjust bad debt" and "cancellations" fields (InvoiceNo starting with "A" and "C" respectively)
 
- 2.1.2 Identify fields with negative quantities that are not "cancellations"

2.2 Check for UnitPrice = 0


2.3 Check for missing values

, duplicates, anomalies
- flag sales with InvoiceNo starting with "A" and "C" (bad debt adjustment and cancellations)?

#### 2.1 Check for Negative UnitPrice and Quantity

In [83]:
# negative price check

negative_price = df[df["UnitPrice"] < 0]

negative_price

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,BadDebt,Cancellation,CustomerID_NA,Description_NA
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,True,False,True,False
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,True,False,True,False


In [84]:
# "Adjust bad debt" records

df.loc[df['InvoiceNo'].str.startswith("A", na = False), :]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,BadDebt,Cancellation,CustomerID_NA,Description_NA
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom,True,False,True,False
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,True,False,True,False
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,True,False,True,False


#### 2.1.1 Identify and Flag Cancellations and Bad Debt Adjustments

In [85]:
# InvoiceNo starting with 'C' indicates cancellation

df.loc[df['InvoiceNo'].str.startswith("C", na = False), :]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,BadDebt,Cancellation,CustomerID_NA,Description_NA
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,False,True,False,False
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,False,True,False,False
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,False,True,False,False
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,False,True,False,False
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,False,True,False,False
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,False,True,False,False
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,False,True,False,False
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,False,True,False,False


In [86]:
# check for other InvoiceNo starting with a letter

df.loc[df["InvoiceNo"].str.contains(r"^[A-Za-z]", na=False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,BadDebt,Cancellation,CustomerID_NA,Description_NA
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,False,True,False,False
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,False,True,False,False
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,False,True,False,False
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,False,True,False,False
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,False,True,False,False
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,False,True,False,False
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,False,True,False,False
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,False,True,False,False


In [87]:
# flag bad debt fields and cancellations

df['BadDebt'] = df['InvoiceNo'].str.contains(r"^A", na=False)
df['Cancellation'] = df['InvoiceNo'].str.contains(r"^C", na=False)


In [88]:
# negative quantity check

negative_quantity = df[df["Quantity"] < 0]

negative_quantity

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,BadDebt,Cancellation,CustomerID_NA,Description_NA
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,False,True,False,False
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,False,True,False,False
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,False,True,False,False
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,False,True,False,False
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,False,True,False,False
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,False,True,False,False
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,False,True,False,False
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,False,True,False,False


#### 2.1.2 Identify Negative Quantities that are NOT Cancellations

In [105]:
# check negative quantity for transactions that are not cancellations

noncancel_negquantity = negative_quantity.loc[~negative_quantity['InvoiceNo'].str.contains(r"^C", na=False)]

# transactions with negative quantities that are not cancellations
# lots of damages and adjustments

noncancel_negquantity['Description'].value_counts()

Description
check                    120
damages                   45
damaged                   42
?                         41
sold as set on dotcom     20
                        ... 
lost??                     1
wet                        1
wet boxes                  1
????damages????            1
lost                       1
Name: count, Length: 138, dtype: int64

In [106]:
noncancel_negquantity['Description'].value_counts().index.tolist()

['check',
 'damages',
 'damaged',
 '?',
 'sold as set on dotcom',
 'Damaged',
 'thrown away',
 'Unsaleable, destroyed.',
 '??',
 'damages?',
 'ebay',
 'wet damaged',
 'smashed',
 'CHECK',
 'missing',
 'wet pallet',
 'Dotcom sales',
 'reverse 21/5/10 adjustment',
 'mixed up',
 'incorrect stock entry.',
 'adjustment',
 'wet rusty',
 'counted',
 'printing smudges/thrown away',
 'crushed',
 'wet/rusty',
 'sold as 1',
 '?missing',
 'stock check',
 'damages wax',
 'dotcom',
 '???missing',
 'rusty throw away',
 'faulty',
 'Dotcom',
 'Given away',
 'mouldy, thrown away.',
 'label mix up',
 'showroom',
 'MIA',
 'samples/damages',
 'incorrectly made-thrown away.',
 'mystery! Only ever imported 1800',
 'sold as 22467',
 'cracked',
 'sold in set?',
 're dotcom quick fix.',
 "Dotcom sold in 6's",
 "thrown away-can't sell.",
 "thrown away-can't sell",
 'wrong code',
 'adjust',
 'POSSIBLE DAMAGES OR LOST?',
 'MERCHANT CHANDLER CREDIT ERROR, STO',
 'Display',
 'Missing',
 'wrong code?',
 'DAMAGED',
 '

In [114]:
a = df.loc[df['InvoiceNo'].str.startswith("A", na = False), :] # bad debt
b = df.loc[df['InvoiceNo'].str.startswith("C", na = False), :] # cancellations

# Summary

print(f"There are {a.shape[0]} fields with bad debt adjustments.")
print(f"There are {b.shape[0]} fields with cancellations.")
print(f"There are {noncancel_negquantity.shape[0]} fields with negative quantities that are not cancellations")

There are 3 fields with bad debt adjustments.
There are 9288 fields with cancellations.
There are 1336 fields with negative quantities that are not cancellations


#### 2.2 Check for UnitPrice = 0

In [123]:
df.loc[(df['UnitPrice'] == 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,BadDebt,Cancellation,CustomerID_NA,Description_NA
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom,False,False,True,True
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom,False,False,True,True
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom,False,False,True,True
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom,False,False,True,True
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom,False,False,True,True
538504,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom,False,False,True,False
538505,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,NaN,United Kingdom,False,False,True,False
538554,581408,85175,NaN,20,2011-12-08 14:06:00,0.0,NaN,United Kingdom,False,False,True,True


In [131]:
print(df.loc[(df['UnitPrice'] == 0) & (df['CustomerID_NA'] == False)].shape[0])
print(df.loc[(df['UnitPrice'] == 0) & (df['Description_NA'] == False)].shape[0])

# Only 40 instances of UnitPrice == 0 occurs when CustomerID is not NA
# 1061 instances of UnitPrice == 0 occurs when Description is not NA
# Consider removing fields with UnitPrice == 0?

40
1061


#### 2.3 Missing values

In [56]:
df.isnull().sum()

# large quantity of null values for 'Description' and 'CustomerID'

InvoiceNo            0
StockCode            0
Description       1454
Quantity             0
InvoiceDate          0
UnitPrice            0
CustomerID      135080
Country              0
BadDebt              0
Cancellation         0
dtype: int64

In [57]:
# # Investigate missing CustomerID

# Missing_CustomerID = data[data["CustomerID"].isna()]

# Missing_CustomerID.head()


In [58]:
# flag missing CustomerID and Description

df["CustomerID_NA"] = df["CustomerID"].isna()

df["Description_NA"] = df["Description"].isna()

df[["CustomerID_NA", "Description_NA"]]



,CustomerID_NA,Description_NA
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
541904,False,False
541905,False,False
541906,False,False
541907,False,False


In [59]:
df['Customer_ID_NA'].value_counts()

KeyError: 'Customer_ID_NA'

In [ ]:
df['Description_NA'].value_counts()

Description_NA
False    540455
True       1454
Name: count, dtype: int64

#### CustomerID analysis

In [123]:
df["CustomerID"].value_counts().reset_index()


,CustomerID,count
0,17841.0,7983
1,14911.0,5903
2,14096.0,5128
3,12748.0,4642
4,14606.0,2782
...,...,...
4367,17986.0,1
4368,13256.0,1
4369,18184.0,1
4370,13747.0,1


In [122]:
mean = df["CustomerID"].value_counts().mean()
stddev = df["CustomerID"].value_counts().std()

print("Average orders per CustomerID:", mean, "\n" "Standard deviation: ", stddev)

Average orders per CustomerID: 93.05329368709972 
Standard deviation:  232.4716077416496
